In [3]:
%pip install tensorflow
%pip install pandas 
%pip install numpy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
%load_ext autoreload

In [5]:
%autoreload 2
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import datetime

from data_handler import LocalToLargeDataLoader, DataLoader

In [ ]:
data_loader: DataLoader = LocalToLargeDataLoader(print_progress=True)
features, y = data_loader.load_raw_data()
test_features, y_raw_test = data_loader.load_raw_test_data()

In [16]:
print(features.head())
feature_array = np.stack(features)
test_feature_array = np.stack(test_features)
print(feature_array.shape)

x_train = tf.convert_to_tensor(feature_array, dtype=tf.float32)
x_test = tf.convert_to_tensor(test_feature_array, dtype=tf.float32)

                 time    cog   sog  rot  heading  navstat  \
0 2024-01-01 00:00:25  284.0   0.7    0       88        0   
1 2024-01-01 00:00:36  109.6   0.0   -6      347        1   
2 2024-01-01 00:01:45  111.0  11.0    0      112        0   
3 2024-01-01 00:03:11   96.4   0.0    0      142        1   
4 2024-01-01 00:03:51  214.0  19.7    0      215        0   

                   vesselId                    portId           etaParsed  
0  61e9f3a8b937134a3c4bfdf7  61d371c43aeaecc07011a37f 2024-01-09 23:00:00  
1  61e9f3d4b937134a3c4bff1f  634c4de270937fc01c3a7689 2024-12-29 20:00:00  
2  61e9f436b937134a3c4c0131  61d3847bb7b7526e1adf3d19 2024-01-02 09:00:00  
3  61e9f3b4b937134a3c4bfe77  61d36f770a1807568ff9a126 2024-12-31 20:00:00  
4  61e9f41bb937134a3c4c0087  634c4de270937fc01c3a74f3 2024-01-25 12:00:00  
(9,)


ValueError: could not convert string to float: 'time'

In [ ]:
y_train = np.array(y)
y_test = np.array(y_raw_test["Evaluation"].apply(score_to_label.label_of_score).to_list())

input_shape = x_test[0].shape
print(f"Input shape {input_shape}")

In [ ]:
parameters=8*8*8
layers=5

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=input_shape),
  tf.keras.layers.Dense(parameters//layers+1, activation='relu'),
  tf.keras.layers.Dense(parameters//layers+1, activation='relu'),
  tf.keras.layers.Dense(parameters//layers+1, activation='relu'),
  tf.keras.layers.Dense(parameters//layers+1, activation='relu'),
  tf.keras.layers.Dense(parameters//layers+1, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1, activation='linear')
])

model.compile(optimizer='adam',
  loss='mean_squared_error',
  metrics=['mae','mape'])

model.summary()

early_stopping = EarlyStopping(
    monitor='val_loss',  # You can change this to 'val_accuracy' or another metric
    patience=5,          # Number of epochs to wait for improvement
    restore_best_weights=True  # Restore model weights from the epoch with the best value of the monitored quantity
)

checkpoint = ModelCheckpoint(
    filepath='modelepoch{epoch:02d}val_loss{val_loss:.2f}.keras',
    monitor='val_loss',
    save_best_only=True,
    save_weights_only=False,
    mode='min',
    verbose=1
)

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
